In [ ]:
from pathlib import Path
import requests, copy
import tiktoken

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import TextLoader   #  텍스트 로더 : For "This model's maximum context length is 16385 tokens" error
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.memory import ConversationBufferMemory


RAW_URL = "https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw"
files_dir = Path("./files")
files_dir.mkdir(parents=True, exist_ok=True)
doc_path = files_dir / "chapter_1.txt"

if not doc_path.exists():
    r = requests.get(RAW_URL, timeout=30)
    r.raise_for_status()
    doc_path.write_text(r.text, encoding="utf-8")
    print("Download complete.")

llm = ChatOpenAI(temperature=0.1, max_tokens=512)


loader = TextLoader(str(doc_path), encoding="utf-8")
raw_docs = loader.load()
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=180,   
    chunk_overlap=30,
)
docs = splitter.split_documents(raw_docs)

cache_dir = LocalFileStore("./.cache/")
emb = OpenAIEmbeddings()
cached_emb = CacheBackedEmbeddings.from_bytes_store(emb, cache_dir)
vs = FAISS.from_documents(docs, cached_emb)
retriever = vs.as_retriever(search_kwargs={"k": 1})   # ✅ 한 번에 1개 청크만

memory = ConversationBufferMemory(
    memory_key="history",
    input_key="question",   
    output_key="answer",    
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer only using the context below.\n"
            "If you don't know the answer, say you don't know.\n\n"
            "Context:\n{context}"
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False,
    output_key="answer",
)

stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
)

model = tiktoken.encoding_for_model("gpt-4o-mini")

def shrink_docs_to_tokens(docs, budget_tokens=2200):
    out, used = [], 0
    for d in docs:
        toks = model.encode(d.page_content)
        n = len(toks)
        if used + n <= budget_tokens:
            out.append(d)
            used += n
        else:
            remain = budget_tokens - used
            if remain > 0:
                d2 = copy.deepcopy(d)
                d2.page_content = model.decode(toks[:remain])
                out.append(d2)
            break
    return out

from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm, chain_type="map_reduce")

def ask(question: str) -> str:
    docs = retriever.get_relevant_documents(question)
    return qa_chain.run(input_documents=docs, question=question)


Created a chunk of size 1078, which is longer than the specified 180
Created a chunk of size 1281, which is longer than the specified 180
Created a chunk of size 375, which is longer than the specified 180
Created a chunk of size 221, which is longer than the specified 180
Created a chunk of size 425, which is longer than the specified 180
Created a chunk of size 1257, which is longer than the specified 180
Created a chunk of size 661, which is longer than the specified 180
Created a chunk of size 355, which is longer than the specified 180
Created a chunk of size 361, which is longer than the specified 180
Created a chunk of size 425, which is longer than the specified 180
Created a chunk of size 1257, which is longer than the specified 180
Created a chunk of size 661, which is longer than the specified 180
Created a chunk of size 323, which is longer than the specified 180
Created a chunk of size 347, which is longer than the specified 180
Created a chunk of size 198, which is longer

In [36]:
if __name__ == "__main__":
    qs = [
        "Aaronson 은 유죄인가요?",
        "그가 테이블에 어떤 메시지를 썼나요?",
        "Julia 는 누구인가요?",
    ]
    for q in qs:
        print("Q:", q)
        print("A:", ask(q))

Q: Aaronson 은 유죄인가요?
A: 예, Jones, Aaronson, 그리고 Rutherford은 범죄를 저질렀다고 여겨졌습니다. 그러나 문서에서 Aaronson이 유죄로 판결받았는지에 대한 구체적인 정보는 제공되지 않았습니다.
Q: 그가 테이블에 어떤 메시지를 썼나요?
A: 해당 문서에서는 그가 테이블에 어떤 메시지를 썼는지에 대한 정보가 제공되지 않습니다.
Q: Julia 는 누구인가요?
A: 'Julia'는 문맥에서 명확히 언급되지 않았기 때문에 정확한 정보를 제공할 수 없습니다.
